In [1]:
from src.Voice_Based_Disease_Screening_using_ML.model_trainer import ModelTrainer

In [4]:
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor,AdaBoostClassifier
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
#from catboost import CatBoostRegressor
#from xgboost import XGBRegressor

In [ ]:
from utils import evaluate_models
from src.Voice_Based_Disease_Screening_using_ML.data_preprocessing import X_train,X_test,y_train,y_test,X_test_scaled,X_train_scaled

In [12]:
def metrics_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [14]:
models = {
    "Linear Regression": LinearRegression(),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "K-Neighbors Regressor": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor(),
    #"XGBRegressor": XGBRegressor(), 
    #"CatBoosting Regressor": CatBoostRegressor(verbose=False),
    "AdaBoost Regressor": AdaBoostRegressor()
}
params={
                "Decision Tree": {
                    'criterion':['squared_error', 'friedman_mse', 'absolute_error', 'poisson'],
                    # 'splitter':['best','random'],
                    # 'max_features':['sqrt','log2'],
                },
                "Lasso": {"alpha": [0.1, 1, 10]},
                "Random Forest":{
                    # 'criterion':['squared_error', 'friedman_mse', 'absolute_error', 'poisson'],
                 
                    # 'max_features':['sqrt','log2',None],
                    'n_estimators': [8,16,32,64,128,256]
                },
                "Gradient Boosting":{
                    # 'loss':['squared_error', 'huber', 'absolute_error', 'quantile'],
                    'learning_rate':[.1,.01,.05,.001],
                    'subsample':[0.6,0.7,0.75,0.8,0.85,0.9],
                    # 'criterion':['squared_error', 'friedman_mse'],
                    # 'max_features':['auto','sqrt','log2'],
                    'n_estimators': [8,16,32,64,128,256]
                },
                "Linear Regression":{},
                #"XGBRegressor":{
                    #'learning_rate':[.1,.01,.05,.001],
                    #'n_estimators': [8,16,32,64,128,256]
                #},
                #"CatBoosting Regressor":{
                    #'depth': [6,8,10],
                    #'learning_rate': [0.01, 0.05, 0.1],
                    #'iterations': [30, 50, 100]
                #},
                "AdaBoost Regressor":{
                    'learning_rate':[.1,.01,0.5,.001],
                    # 'loss':['linear','square','exponential'],
                    'n_estimators': [8,16,32,64,128,256]
                },
                "Ridge": {"alpha": [0.1, 1, 10]},
                "K-Neighbors Regressor": {"n_neighbors": [3, 5, 7]},
                "Random Forest Regressor": {"n_estimators": [50, 100]},
                "Random Forest Classifier": {
                            "n_estimators": [100],
                            "max_depth": [5, None]
                },
                "AdaBoost Classifier": {
                            "n_estimators": [100],
                            "learning_rate": [0.1, 1.0]
                },
                "CatBoost Classifier": {
                            "iterations": [100],  # Reduce from default 1000
                            "learning_rate": [0.1],
                            "depth": [4, 6]
                }
                
            }
model_list = []
r2_list =[]
model_report:dict=evaluate_models(X_train,y_train,X_test,y_test,models,params)

            ## To get best model score from dict
best_model_score = max(sorted(model_report.values()))

             ## To get best model name from dict

best_model_name = list(model_report.keys())[
                list(model_report.values()).index(best_model_score)
            ]
best_model = models[best_model_name]
print(model_report)
print("This is the best model:")
print(best_model_name)

model_names = list(params.keys())

actual_model=""

for model in model_names:
    if best_model_name == model:
        actual_model = actual_model + model

best_params = params[actual_model]

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train) # Train model

    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Evaluate Train and Test dataset
    model_train_mae , model_train_rmse, model_train_r2 = metrics_model(y_train, y_train_pred)

    model_test_mae , model_test_rmse, model_test_r2 = metrics_model(y_test, y_test_pred)

    
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    
    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))

    print('----------------------------------')
    
    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2))
    r2_list.append(model_test_r2)
    
    print('='*35)
    print('\n')

/Users/vaibhavkavdia/Desktop/Projects_for_Resume/Medical/Voice-Based-Disease-Screening-using-ML/venv/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:953: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/vaibhavkavdia/Desktop/Projects_for_Resume/Medical/Voice-Based-Disease-Screening-using-ML/venv/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 942, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/vaibhavkavdia/Desktop/Projects_for_Resume/Medical/Voice-Based-Disease-Screening-using-ML/venv/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 308, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

ValueError: Classification metrics can't handle a mix of multiclass and continuous targets